In [12]:
import nltk
from collections import deque
# nltk.download('dependency_treebank')
from nltk.corpus import dependency_treebank
import numpy as np

sents = dependency_treebank.parsed_sents()

train_set, test_set = sents[:int(0.9*len(sents))], sents[int(0.9*len(sents)):]

In [15]:
def create_hot_vec(vals):
    n = len(vals)
    vec = np.zeros((n**2,1))
    return vec

def fill_hot_vec(sents, vocab, tags):
    n = len(sents.nodes)
    vocab_hot_vec, tags_hot_vec = create_hot_vec(vocab), create_hot_vec(tags)
    for i in range(1,len(sents.nodes)):
        dest_node = sents.nodes[i]
        source_node = sents.nodes[dest_node['head']]
        source_word_ind,source_tag_ind = vocab[source_node['word']],tags[source_node['tag']]
        dest_word_ind , dest_tag_ind = vocab[dest_node['word']],tags[dest_node['tag']]
        vocab_hot_vec[(source_word_ind-1)*n+(dest_word_ind-1)] = 1
        tags_hot_vec[(source_tag_ind-1)*n+(dest_tag_ind-1)] = 1

    return np.concatenate((vocab_hot_vec,tags_hot_vec),axis=0)

def feature_func(sent):
    tags,vocab = dict(), dict()
    nodes_stack = [sent.nodes[i] for i in range(len(sent.nodes))]
    words_ind,tag_ind =0,0
    for i in range(len(nodes_stack)):
        node = nodes_stack[i]
        if node['word'] not in vocab:
            vocab[node['word']] = words_ind
            words_ind+=1
        if node['tag'] not in tags:
            tags[node['tag']] = tag_ind
            tag_ind+=1
    return fill_hot_vec(sent,vocab,tags)
a = feature_func(train_set[1])

KeyboardInterrupt: 

In [ ]:
LEARNING_RATE = 1
def perceptron(X, y0, max_iter=1000):
    """
    :param X: samples matrix (N x P)
    :param y0: labels vector (P x 1)
    :param max_iter: maximal number of iterations
    :return: [w, converged, epochs], 'w' being the final weights vector, 'converged' as a flag indicating convergence of
    the algorithm and 'epochs' as the number of epochs executed.
    """
    it = 0
    w = np.ones((X.shape[0],))
    changed = True

    while it < max_iter and changed:

        changed = False

        for i in range(X.shape[1]):

            if ((w.T @ X[:, i]) * y0[i]) < 0:
                w += LEARNING_RATE * X[:, i] * y0[i]
                changed = True

        it += 1

    return [w, it < max_iter, it]



In [ ]:
def perceptron(train_sent)